In [30]:
import time
Start = time.time() 
import os
import numpy as np
import pandas as pd
import cv2
import math
import matplotlib.pyplot as plt
from scipy.ndimage.filters import convolve

In [31]:
all_img = [] 
def directory_name(dirname):
    all_img.clear()
    filenumber = len([name for name in os.listdir(dirname) if os.path.isfile(os.path.join(dirname, name))])
    for i in range(1,filenumber+1):
        img = cv2.imread(dirname + "/" + str(i) + ".jpg")
        all_img.append(img)

In [32]:
def SIFT(img):
    #img = cv2.imread(img)
#    def rgb2gray(rgb):return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])
#    image_gray = rgb2gray(img)
    image_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    first_image = convolve(image_gray, gaussian_filter(1.3))
    gaussian_octave = generate_octave(first_image, s=3, sigma=1.6)
    DoG_octave = generate_DoG_octave(gaussian_octave)
    kp = find_keypoints_for_DoG_octave(DoG_octave)
    kp = assign_orientation(kp, DoG_octave)
    local_descriptors = get_local_descriptors(kp, DoG_octave)
    kp_o1 = kp[:,:-2]
    kp_o1 = kp_o1[:,::-1]
    return kp_o1 ,local_descriptors

In [33]:
def gaussian_filter(sigma):    
    size = 2*np.ceil(3*sigma)+1 
    x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1] 
    g = np.exp(-(x**2)/(2*(sigma**2)))/(sigma*math.sqrt(2*math.pi))
    ga=g.sum()
    return g/ga

In [34]:
def generate_octave(first_img_in_octive, s, sigma): 
    octave = [first_img_in_octive] 
    k = 2**(1/s) 
    kernel = gaussian_filter(k * sigma) 
    for _ in range(s+1): 
        next_level = convolve(octave[-1], kernel) 
        octave.append(next_level) 
    return octave
##############################################################################
def generate_DoG_octave(gaussian_octave): 
    octave = [] 
    for i in range(1, len(gaussian_octave)):   
        octave.append(gaussian_octave[i] - gaussian_octave[i-1]) 
    return octave

In [35]:
def find_keypoints_for_DoG_octave(DoG_octave):
    r_th=10
    t_c=0.03
    height,width= DoG_octave[0].shape
    keypoints = []
    for DoG_octave_num in range(1,len(DoG_octave)-1):
        for row in range(1, height-1):
            for col in range(1, width-1):
                center = DoG_octave[DoG_octave_num][row,col]
                other27 = np.vstack([DoG_octave[DoG_octave_num-1][row-1:row+2, col-1:col+2], DoG_octave[DoG_octave_num][row-1:row+2, col-1:col+2],DoG_octave[DoG_octave_num+1][row-1:row+2, col-1:col+2]])
                other9 = DoG_octave[DoG_octave_num][row-1:row+2, col-1:col+2]
                maxValue = other27.max()
                if center == maxValue: 
                    if np.count_nonzero(other9 == center) == 1:
                        dx = (float(DoG_octave[DoG_octave_num][row+1, col]) - float(DoG_octave[DoG_octave_num][row-1, col])) * 0.5
                        dy = (float(DoG_octave[DoG_octave_num][row, col+1]) - float(DoG_octave[DoG_octave_num][row, col-1])) * 0.5 
                        ds = (float(DoG_octave[DoG_octave_num+1][row, col]) - float(DoG_octave[DoG_octave_num-1][row, col])) * 0.5
                        dxx = (float(DoG_octave[DoG_octave_num][row+1, col]) + float(DoG_octave[DoG_octave_num-1][row-1, col]) - 2 * float(DoG_octave[DoG_octave_num][row,col]))        
                        dyy = (float(DoG_octave[DoG_octave_num][row, col+1]) + float(DoG_octave[DoG_octave_num-1][row, col-1]) - 2 * float(DoG_octave[DoG_octave_num][row, col]))   
                        dss = (float(DoG_octave[DoG_octave_num+1][row, col]) + float(DoG_octave[DoG_octave_num-1][row, col]) - 2 * float(DoG_octave[DoG_octave_num][row, col]))
                        dxy = (float(DoG_octave[DoG_octave_num][row+1, col+1]) - float(DoG_octave[DoG_octave_num-1][row-1, col+1]) - float(DoG_octave[DoG_octave_num][row+1, col-1]) + float(DoG_octave[DoG_octave_num][row-1, col-1])) * 0.25 
                        dxs = (float(DoG_octave[DoG_octave_num+1][row+1, col]) - float(DoG_octave[DoG_octave_num+1][row-1, col]) - float(DoG_octave[DoG_octave_num-1][row+1, col]) + float(DoG_octave[DoG_octave_num-1][row-1, col])) * 0.25 
                        dys = (float(DoG_octave[DoG_octave_num+1][row, col+1]) - float(DoG_octave[DoG_octave_num+1][row, col-1]) - float(DoG_octave[DoG_octave_num-1][row, col+1]) + float(DoG_octave[DoG_octave_num-1][row, col-1])) * 0.25 
                        dD = np.matrix([[dx], [dy], [ds]]) 
                        H = np.matrix([[dxx, dxy, dxs], [dxy, dyy, dys], [dxs, dys, dss]])
                        offset = -np.linalg.pinv(H).dot(dD)
                        contrast = (DoG_octave[DoG_octave_num][row, col]) + 0.5 * np.dot(dD.transpose(), offset)
                        if (((dxx + dyy) ** 2) * r_th) > ((dxx * dyy - (dxy ** 2)) * ((r_th + 1) ** 2)):
                            continue
                        if np.absolute(contrast) < t_c:
                            continue
                        keypoints.append(np.array([row, col, DoG_octave_num]))                                                                                                
    return np.array(keypoints)

def assign_orientation(kps, octave, num_bins=36):
    count_i = 0
    height,width= octave[0].shape
    keypoint_info = [] 
    bin_width = 360//num_bins
    magnitude = np.zeros((height, width))
    theta = np.zeros((height, width))
    magnitudes =[]
    thetas = []    
    for i in range(1,len(octave)-1):
        L = octave[i]
        for row in range(1, height-1):
            for col in range(1, width-1 ):
                magnitude[row, col] = ((float((L[row+1, col]) - float(L[row-1, col])) ** 2) + ((float(L[row, col+1]) - float(L[row, col-1])) ** 2) ) ** 0.5   
                theta[row, col] = (36 / (2 * np.pi)) * (np.pi + np.arctan2((float(L[row, col+1]) - float(L[row, col-1])), (float(L[row+1, col]) - float(L[row-1, col]))))
        magnitudes.append(magnitude)
        thetas.append(theta)   
    for kp in kps:
        count_i +=1
        x, y, s = int(kp[0]), int(kp[1]), int(kp[2]) 
        sigma = s*1.5 
        filter_range = int(2*np.ceil(sigma)+1) 
        kernel = gaussian_filter(sigma)
        m = magnitudes[s-1]
        t = thetas[s-1]
        orient_hist = np.zeros([num_bins,1])
        for filter_x in range(-filter_range,filter_range+1):
            for filter_y in range(-filter_range,filter_range+1):
                if  (x + filter_x) < 0 or (x + filter_x > height - 1) or (filter_y + y < 0) or (filter_y + y > width - 1):
                    continue
                weight = kernel[filter_x+filter_range, filter_y+filter_range] * m[x + filter_x,filter_y + y]
                bin_idx = np.clip(np.floor(t[x + filter_x,filter_y + y]), 0, 35)
                orient_hist[int(np.floor(bin_idx))] += weight
        maxval = np.amax(orient_hist)
        maxidx = np.argmax(orient_hist)
        keypoint_info.append([kp[0], kp[1], kp[2], fit_parabola(orient_hist, maxidx, bin_width)])
        orient_hist_new = orient_hist
        orient_hist_new[maxidx] = 0
        newmaxval = np.amax(orient_hist_new)
        if newmaxval > 0.8 * maxval:
            newmaxidx = np.argmax(orient_hist_new)
            keypoint_info.append([kp[0], kp[1], kp[2],fit_parabola(orient_hist, newmaxidx, bin_width)])
    return np.array(keypoint_info)

In [36]:
def get_local_descriptors(keypoints, octave, w=16, num_subregion=4, num_bin=8):
    descs = [] 
    bin_width = 360//num_bin
    for keypoint in keypoints:
        x, y, s = int(keypoint[0]), int(keypoint[1]), int(keypoint[2]) 
        kernel = gaussian_filter(w/6)
        L = octave[s]      
        t, l = max(0, x-w//2), max(0, y-w//2) 
        b, r = min(L.shape[0], x+w//2+1), min(L.shape[1], y+w//2+1)
        patch = L[t:b, l:r]
        height,width= patch.shape
        magnitude = np.zeros((height, width))
        theta = np.zeros((height, width))
        for row in range(1, height - 1):
            for col in range(1, width - 1):
                magnitude[row, col] = ((float((patch[row+1, col]) - float(patch[row-1, col])) ** 2) + ((float(patch[row, col+1]) - float(patch[row, col-1])) ** 2) ) ** 0.5   
                theta[row, col] = (360 / (2 * np.pi)) * (np.pi + np.arctan2((float(patch[row, col+1]) - float(patch[row, col-1])), (float(patch[row+1, col]) - float(patch[row-1, col]))))
        if patch.shape[0] < w+1 :
            if t == 0:
                magnitude = np.pad(magnitude,((kernel.shape[0]-patch.shape[0],0),(0,0)),'constant')
            else: 
                magnitude = np.pad(magnitude,((0,kernel.shape[0]-patch.shape[0]),(0,0)),'constant')
        if patch.shape[1] < w+1: 
            if l == 0:
                magnitude = np.pad(magnitude,((0,0),(kernel.shape[1]-patch.shape[1],0)),'constant')
            else: 
                magnitude = np.pad(magnitude,((0,0),(0,kernel.shape[1]-patch.shape[1])),'constant')
        weight_magnitude = magnitude*kernel       
        subregion_w = w//num_subregion 
        featvec = np.zeros(num_bin * num_subregion**2, dtype=np.float32)       
        for i in range(0, subregion_w): 
            for j in range(0, subregion_w):
                t, l = i*subregion_w, j*subregion_w 
                b, r = min(L.shape[0], (i+1)*subregion_w), min(L.shape[1], (j+1)*subregion_w)
                hist = get_histogram_for_subregion(weight_magnitude[t:b, l:r].ravel(), theta[t:b, l:r].ravel(), num_bin, keypoint[3], bin_width, subregion_w)
                featvec[i*subregion_w*num_bin + j*num_bin:i*subregion_w*num_bin + (j+1)*num_bin] = hist.flatten()
        descs.append(featvec)
    return np.array(descs)

In [37]:
class my_Set:
    def __init__(self, image, kp, ds, shft = np.float32([[1, 0, 0], [0, 1, 0]])):
        self.image = image
        self.kp = kp
        self.ds = ds
        self.shft = shft
def stitch(setB, setA, dirname):
    (imageA, kpsA, featuresA) = (setA.image, setA.kp, setA.ds)
    (imageB, kpsB, featuresB, old_shift) = (setB.image, setB.kp, setB.ds, setB.shft)   
    matcher = cv2.DescriptorMatcher_create("BruteForce")
    rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
    matches = []
    for m in rawMatches:
        if len(m) == 2 and m[0].distance < m[1].distance * 0.8:
            matches.append((m[0].trainIdx, m[0].queryIdx))
    if len(matches) > 4:
        ptsA = np.float32([kpsA[i] for (_, i) in matches])
        ptsB = np.float32([kpsB[i] for (i, _) in matches])
    else:
        plt.figure()
        plt.imshow(imageA)
        plt.show()
        plt.imshow(imageB)
        plt.show()
        raise Exception
        return None 
    M = BF_RANSAC(ptsA, ptsB, imageA.shape[0]/32.) + old_shift - np.float32([[1, 0, 0], [0, 1, 0]])
    result = cv2.warpAffine(imageA,M,(imageA.shape[1] + imageB.shape[1], imageA.shape[0] + imageB.shape[0]))
    (_, cols) = np.where(result[:imageB.shape[0]-1,:imageB.shape[1]-1,0] != 0)
    (maxcols, mincols) = (max(cols), min(cols))
    ratio = np.float32(maxcols - mincols)
    for y in range(imageB.shape[1]-1):
        for x in range(imageB.shape[0]-1):
            if result[x,y].sum() != 0:
                result[x,y] = result[x,y]*((y-mincols)/ratio) + imageB[x,y]*((ratio-y+mincols)/ratio)
            else:
                result[x,y] = imageB[x,y]
    newSet = my_Set(crop(result), kpsA, featuresA, M)
    return newSet

In [38]:
def alignRecover(myset):
    (image, shift) = (myset.image, myset.shft)
    (x, y) = shift[0,2], shift[1,2]
    yabs = np.abs(y)
    if x*y < 0:
        col_shift = np.linspace(yabs, 0, num=image.shape[1], dtype=np.uint16)
    else:
        col_shift = np.linspace(0, yabs, num=image.shape[1], dtype=np.uint16)
    aligned = np.zeros((image.shape[0] - np.uint16(yabs), image.shape[1], image.shape[2]), dtype = np.uint8)
    for x in range(image.shape[1]):
        aligned[:,x] = image[col_shift[x]:image.shape[0] - np.uint16(yabs) + col_shift[x],x]
    return crop(aligned)
def generate_gaussian_pyramid(img, num_octave, s, sigma):
    pyr = []
    for _ in range(num_octave):
        octave = generate_octave(img, s, sigma)
        pyr.append(octave)
        img = cv2.resize(octave[-3], (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_NEAREST)
    return pyr
def generate_DoG_pyramid(gaussian_pyramid):
    pyr = [] 
    for gaussian_octave in gaussian_pyramid: 
        pyr.append(generate_DoG_octave(gaussian_octave)) 
    return pyr
def get_keypoints(DoG_pyr):
    kps = [] 
    for DoG_octave in DoG_pyr: 
        kps.append(find_keypoints_for_DoG_octave(DoG_octave)) 
    return kps

In [39]:
def fit_parabola(hist, binno, bin_width): 
    centerval = binno*bin_width + bin_width/2. 
    if binno == len(hist)-1: rightval = 360 + bin_width/2. 
    else: rightval = (binno+1)*bin_width + bin_width/2. 
    if binno == 0: leftval = -bin_width/2. 
    else: leftval = (binno-1)*bin_width + bin_width/2. 
    A = np.array([[centerval**2, centerval, 1], [rightval**2, rightval, 1], [leftval**2, leftval, 1]]) 
    b = np.array([ hist[binno], hist[(binno+1)%len(hist)], hist[(binno-1)%len(hist)]])
    x = np.linalg.lstsq(A, b, rcond=None)[0]
    if x[0] == 0: x[0] = 1e-6 
    return -x[1]/(2*x[0])
def assign_orientation(kps, octave, num_bins=36):
    count_i = 0
    height,width= octave[0].shape
    keypoint_info = [] 
    bin_width = 360//num_bins
    magnitude = np.zeros((height, width))
    theta = np.zeros((height, width))
    magnitudes =[]
    thetas = []   
    for i in range(1,len(octave)-1):
        L = octave[i]
        for row in range(1, height-1):
            for col in range(1, width-1 ):
                magnitude[row, col] = ((float((L[row+1, col]) - float(L[row-1, col])) ** 2) + ((float(L[row, col+1]) - float(L[row, col-1])) ** 2) ) ** 0.5   
                theta[row, col] = (36 / (2 * np.pi)) * (np.pi + np.arctan2((float(L[row, col+1]) - float(L[row, col-1])), (float(L[row+1, col]) - float(L[row-1, col]))))
        magnitudes.append(magnitude)
        thetas.append(theta)   
    for kp in kps:
        count_i +=1
        x, y, s = int(kp[0]), int(kp[1]), int(kp[2]) 
        sigma = s*1.5 
        filter_range = int(2*np.ceil(sigma)+1) 
        kernel = gaussian_filter(sigma)
        m = magnitudes[s-1]
        t = thetas[s-1]
        orient_hist = np.zeros([num_bins,1])
        for filter_x in range(-filter_range,filter_range+1):
            for filter_y in range(-filter_range,filter_range+1):
                if  (x + filter_x) < 0 or (x + filter_x > height - 1) or (filter_y + y < 0) or (filter_y + y > width - 1):
                    continue
                weight = kernel[filter_x+filter_range, filter_y+filter_range] * m[x + filter_x,filter_y + y]
                bin_idx = np.clip(np.floor(t[x + filter_x,filter_y + y]), 0, 35)
                orient_hist[int(np.floor(bin_idx))] += weight
        maxval = np.amax(orient_hist)
        maxidx = np.argmax(orient_hist)
        keypoint_info.append([kp[0], kp[1], kp[2], fit_parabola(orient_hist, maxidx, bin_width)])
        orient_hist_new = orient_hist
        orient_hist_new[maxidx] = 0
        newmaxval = np.amax(orient_hist_new)
        if newmaxval > 0.8 * maxval:
            newmaxidx = np.argmax(orient_hist_new)
            keypoint_info.append([kp[0], kp[1], kp[2],fit_parabola(orient_hist, newmaxidx, bin_width)])
    return np.array(keypoint_info)

In [ ]:
def get_histogram_for_subregion(m, theta, num_bin, reference_angle, bin_width, subregion_w): 
    bin_width = 360//num_bin
    orient_hist = np.zeros([num_bin,1]) 
    c = subregion_w/2 - .5
    for i, (mag, angle) in enumerate(zip(m, theta)):
        angle = (angle-reference_angle) % 360
        binno = np.clip(np.floor(np.floor(angle)//bin_width), 0, 7)
        vote = mag
        
        orient_hist[int(binno)] += vote
    return orient_hist
def BF_RANSAC(srcPts, dstPts, yrange):
    m = np.float32([(srcPts[i], dstPts[i]) for i in range(len(srcPts))])
    best_shift = [0, 0]
    max_inliner = 0
    for i in range(len(m)):
        sample = m[i]
        shift = sample[1] - sample[0]
        if shift[1] > yrange:
            continue       
        tstPts = m[:,0] + shift
        distance = m[:,1] - tstPts
        inliner = 0
        for d in distance:
            if np.sqrt((d**2).sum()) < 4:
                inliner = inliner + 1    
        if inliner > max_inliner:
            max_inliner = inliner
            best_shift = shift
        elif inliner == max_inliner:
            best_shift = (best_shift + shift) / 2
    return np.float32([[1, 0, best_shift[0]], [0, 1, best_shift[1]]])
def crop(image):
    (rows, cols) = np.where(image[:,:,0] + image[:,:,1] + image[:,:,2] > 0)
    x, y, w, h = min(cols), min(rows), max(cols), max(rows)
    result = image[y:y+h,x:x+w]
    return result
f = open('testfile.txt', 'r')
dirname = str(f.readline()).strip()
while(dirname):
    array_img = []
    directory_name(dirname)  
    i = 1
    for img in all_img:
        (kp, ds) = SIFT(img)
        newSet = my_Set(img, kp, ds)
        array_img.append(newSet)
        i += 1
    i = 1
    while(len(array_img) > 1):
        newSet = stitch(array_img[0], array_img[1],dirname)
        if newSet is None:
            break
        array_img = array_img[1:]
        array_img[0] = newSet
        i += 1    
    imageout = alignRecover(array_img[0]) 
    plt.figure()
    plt.imshow(imageout)
    cv2.imwrite(dirname + ".jpg", imageout)
    dirname = str(f.readline()).strip()    
End = time.time() 
print("Total %f sec" % (End - Start))    